# Assemble Program

In [1]:
var x86Assembler = require("../dist/x86Assembler/src/assembler")

In [2]:
// Create Assembler object
var assembler = new x86Assembler.Assembler();

In [3]:
// Define code to be compiled
var code = `
    mov $0x20,%ax
    mov $0xa,%bx
    add %bx,%ax
    and $0xff00,%ax
    xor %bx,%bx
    inc %bx
`

In [4]:
// Assemble this code 
var assembled = assembler.assembleProgram(code);

// Print out the assembled code in tabular manner
console.log(assembled.toString())

Addr			Machine Code			Assembly
--------------------------------------------------------------------------------
0:			66 b8 20 00                   mov  $0x20,%ax
4:			66 bb 0a 00                   mov  $0xa,%bx
8:			66 01 d8                      add  %bx,%ax
b:			66 25 00 ff                   and  $0xff00,%ax
f:			66 31 db                      xor  %bx,%bx
12:			66 43                         inc  %bx



In [5]:
// Get machine code 
var machineCode = assembled.getMachineCode()
console.log(machineCode)

Uint8Array(20) [
  102, 184,  32,   0, 102, 187,
   10,   0, 102,   1, 216, 102,
   37,   0, 255, 102,  49, 219,
  102,  67
]


In [6]:
// Get metadat to be used for Debugging in PC
var metadata = assembled.genMetadata()
console.log(metadata)

{
  program: '\n' +
    '    mov $0x20,%ax\n' +
    '    mov $0xa,%bx\n' +
    '    add %bx,%ax\n' +
    '    and $0xff00,%ax\n' +
    '    xor %bx,%bx\n' +
    '    inc %bx\n',
  line_nums: [ 2, 3, 4, 5, 6, 7 ],
  symbol_table: {},
  reverse_symbol_table: {},
  instruction_start_addr: [ 0, 4, 8, 11, 15, 18 ],
  instruction_lengths: [ 4, 4, 3, 4, 3, 2 ],
  addr_instruction_map: { '0': 0, '4': 1, '8': 2, '11': 3, '15': 4, '18': 5 }
}


# Load Code in PC

In [10]:
var PC = require("../dist/x86PC/src/pc")

In [11]:
// Create PC with 256 Bytes of memory
var pc = new PC.PC(256)

// Some utility functions
function showIP(pc) {
    console.log("Instruction Pointer : 0x" +pc.cpu.getInstructionPointer().toString(16))
}
function showRegister(pc, reg) {
    console.log("Register "+ reg + " : " +pc.cpu.registers[reg].getHexadecimalBytes().join(" "))
}

In [12]:
// Load the Machine code in PC starting at address 0
pc.loadAssembledProgram(machineCode, 0)
showIP(pc)

Instruction Pointer : 0x0


In [13]:
// Execute the next instruction
pc.executeNextInstruction()
showIP(pc)
showRegister(pc, "eax")

mov $0x20, %ax
Instruction Pointer : 0x4
Register eax : 0 0 0 20


In [14]:
// Execute the next instruction
pc.executeNextInstruction()
showIP(pc)
showRegister(pc, "ebx")

mov $0xa, %bx
Instruction Pointer : 0x8
Register ebx : 0 0 0 a


In [15]:
// Execute the next instruction
pc.executeNextInstruction()
showIP(pc)
showRegister(pc, "eax")
showRegister(pc, "ebx")

add %bx, %ax
Instruction Pointer : 0xb
Register eax : 0 0 0 2a
Register ebx : 0 0 0 a


In [16]:
// Execute the next instruction
pc.executeNextInstruction()
showIP(pc)
showRegister(pc, "eax")

and $0xff00, %ax
Instruction Pointer : 0xf
Register eax : 0 0 0 0


In [17]:
// Execute the next instruction
pc.executeNextInstruction()
showIP(pc)
showRegister(pc, "ebx")

xor %bx, %bx
Instruction Pointer : 0x12
Register ebx : 0 0 0 0


In [18]:
// Execute the next instruction
pc.executeNextInstruction()
showIP(pc)
showRegister(pc, "ebx")

inc %bx
Instruction Pointer : 0x14
Register ebx : 0 0 0 1


In [19]:
// No more instructions
pc.executeNextInstruction()

false